In [1]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import pandas as pd

In [2]:
main_link = 'https://hh.ru'

search_vacancy = 'data scientist'
page = 0
while True:
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'}

    params = {'text': search_vacancy,
              'clusters': 'true',
              'enable_snippets': 'true',
              'salary': '',
              'st': 'searchVacancy',
              'fromSearch': 'true',
              'page': page}

    response = requests.get(main_link + '/search/vacancy/', params=params, headers=header)
    soup = bs(response.text, 'html.parser')

    vacancy_block = soup.find('div', {'class': 'vacancy-serp'})

    vacancy_list = vacancy_block.findChildren(recursive=False)

    vacancies = []

    for vacancy in vacancy_list:
        if vacancy['class'] == ['vacancy-serp-item'] or vacancy['class'] == ['vacancy-serp-item', 'vacancy-serp-item_premium']:
            link = vacancy.find('a', {'class': 'bloko-link HH-LinkModifier'})['href']
            company = vacancy.find('a', {'class': 'bloko-link bloko-link_secondary'}).text
            city = vacancy.find('span', {'class': 'vacancy-serp-item__meta-info'}).text
            name = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-title'}).text
            try:
                salary = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}).text
            except AttributeError:
                salary = 'None'
            salary = salary.replace('\xa0', '')

            vacancy_data = {'link': link,
                            'company': company,
                            'city': city,
                            'name': name,
                            'salary': salary}

            vacancies.append(vacancy_data)

    page += 1

    if soup.find('a', {'class': 'bloko-button HH-Pager-Controls-Next HH-Pager-Control'}) is None:
        break


In [3]:
df_vacancies = pd.DataFrame(vacancies)

In [4]:
df_vacancies.head(5)

,link,company,city,name,salary
0,https://hh.ru/vacancy/36579115?query=data%20sc...,HUAWEI,"Санкт-Петербург, Пушкинская и еще 1 \n ...",Mathematician (Data scientist),None
1,https://hh.ru/vacancy/36999371?query=data%20sc...,ООО Цифроматика,"Санкт-Петербург, Площадь Восстания\n ...",Data Scientist,до 160000 руб.
2,https://hh.ru/vacancy/36859214?query=data%20sc...,Grid Dynamics,Саратов\n Показать ...,Senior Data Scientist,None
3,https://hh.ru/vacancy/36966964?query=data%20sc...,Recruitment Boutique S.M.Art,Москва\n Показать н...,Data scientist,None
4,https://hh.ru/vacancy/36694503?query=data%20sc...,"ПАО МегаФон, Офис","Москва, Чеховская и еще 2 \n ...",Senior data scientist,None
